#### Renan Felippe de Almeida Barros
#### Date: 03/26/2025
#### Summary: 
##### LLM Document Analysis

In [1]:
import pandas as pd
import requests

In [2]:
headers = {"User-Agent": "re514236@ucf.edu"}  # Need to add your email address here

In [3]:
def cik_matching_ticker(ticker, headers=headers):
    ticker = ticker.upper().replace(".", "-")
    ticker_json = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=headers
    ).json()

    for company in ticker_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10)
            return cik
    raise ValueError(f"Ticker {ticker} not found in SEC database")

In [4]:
cik_matching_ticker("wsm")

'0000719955'

In [5]:
def cik_matching_ticker(ticker, headers=headers):
    ticker = ticker.upper().replace(".", "-")
    ticker_json = requests.get(
        "https://www.sec.gov/files/company_tickers.json", headers=headers
    ).json()

    for company in ticker_json.values():
        if company["ticker"] == ticker:
            cik = str(company["cik_str"]).zfill(10)
            return cik
    raise ValueError(f"Ticker {ticker} not found in SEC database")

In [6]:
def get_submission_data_for_ticker(ticker, headers=headers, only_filings_df=False):
    """
    Get the data in json form for a given ticker. For example: 'cik', 'entityType', 'sic', 'sicDescription', 'insiderTransactionForOwnerExists', 'insiderTransactionForIssuerExists', 'name', 'tickers', 'exchanges', 'ein', 'description', 'website', 'investorWebsite', 'category', 'fiscalYearEnd', 'stateOfIncorporation', 'stateOfIncorporationDescription', 'addresses', 'phone', 'flags', 'formerNames', 'filings'

    Args:
        ticker (str): The ticker symbol of the company.

    Returns:
        json: The submissions for the company.

    Raises:
        ValueError: If ticker is not a string.
    """
    cik = cik_matching_ticker(ticker)
    headers = headers
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    company_json = requests.get(url, headers=headers).json()
    if only_filings_df:
        return pd.DataFrame(company_json["filings"]["recent"])
    else:
        return company_json

In [7]:
def get_8k_filings(ticker, just_accession_numbers=False, headers=headers):
    """
    Fetch recent 8-K filings for a given ticker.

    Args:
        ticker (str): The ticker symbol of the company.
        just_accession_numbers (bool): If True, return only accession numbers.
        headers (dict): Headers for SEC requests.

    Returns:
        DataFrame: A DataFrame containing recent 8-K filings.
    """
    # Fetch submission data
    company_filings_df = get_submission_data_for_ticker(
        ticker, only_filings_df=True, headers=headers
    )
    
    # Filter for 8-K forms
    df = company_filings_df[company_filings_df["form"] == "8-K"]
    
    # Return only accession numbers if requested
    if just_accession_numbers:
        df = df.set_index("reportDate")
        accession_df = df["accessionNumber"]
        return accession_df
    else:
        return df

In [8]:
ticker = "WSM"
eight_k_filings = get_8k_filings(ticker)
eight_k_filings

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,core_type,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
6,0001628280-25-013687,2025-03-19,2025-03-19,2025-03-19T13:09:22.000Z,34,8-K,001-14077,25750977,"2.02,8.01,9.01",XBRL,629716,1,1,wsm-20250319.htm,8-K
27,0001628280-24-048597,2024-11-20,2024-11-20,2024-11-20T13:05:22.000Z,34,8-K,001-14077,241478326,"2.02,9.01",XBRL,538235,1,1,wsm-20241120.htm,8-K
42,0001628280-24-041493,2024-09-27,2024-09-25,2024-09-27T21:12:11.000Z,34,8-K,001-14077,241335943,"5.03,9.01",XBRL,545053,1,1,wsm-20240925.htm,8-K
49,0001628280-24-038109,2024-08-22,2024-08-22,2024-08-22T12:03:01.000Z,34,8-K,001-14077,241230161,"2.02,9.01",XBRL,534940,1,1,wsm-20240822.htm,8-K
53,0001628280-24-034200,2024-08-01,2024-08-01,2024-08-01T20:54:06.000Z,34,8-K,001-14077,241167570,"5.02,9.01",XBRL,174741,1,1,wsm-20240801.htm,8-K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,0001193125-12-363503,2012-08-21,2012-08-21,2012-08-21T20:06:01.000Z,34,8-K,001-14077,121047838,"2.02,9.01",8-K,374010,0,0,d400010d8k.htm,FORM 8-K
969,0001193125-12-319693,2012-07-27,2012-07-27,2012-07-27T20:53:24.000Z,34,8-K,001-14077,12991191,"5.02,9.01",8-K,35623,0,0,d386646d8k.htm,FORM 8-K
973,0001193125-12-287672,2012-06-28,2012-06-22,2012-06-28T20:02:55.000Z,34,8-K,001-14077,12932682,1.01,8-K,15875,0,0,d373456d8k.htm,FORM 8-K
986,0001193125-12-249197,2012-05-25,2012-05-24,2012-05-25T20:02:57.000Z,34,8-K,001-14077,12871346,"5.02,5.07,9.01",8-K,53134,0,0,d359080d8k.htm,FORM 8-K


In [9]:
import requests
import pandas as pd
import ollama
import json
import time
from bs4 import BeautifulSoup
 
# Maps tickers -> real company names
TICKER_COMPANY_MAP = {
    "AAPL": "Apple Inc.",
    "MSFT": "Microsoft Corporation",
    "TSLA": "Tesla, Inc.",
    "AMZN": "Amazon.com, Inc.",
    "GOOGL": "Alphabet Inc.",
    "NKE": "NIKE, Inc.",
    "NVDA": "NVIDIA Corporation",
    "LMT": "Lockheed Martin Corporation",
    "META": "Meta Platforms, Inc.",
    "INTC": "Intel Corporation",
    "AMD": "Advanced Micro Devices, Inc.",
    "NFLX": "Netflix, Inc.",
    "CRM": "Salesforce, Inc.",
    "ADBE": "Adobe Inc.",
    "PYPL": "PayPal Holdings, Inc.",
    "UBER": "Uber Technologies, Inc.",
    "ROKU": "Roku, Inc.",
    "SBUX": "Starbucks Corporation",
    "PFE": "Pfizer Inc.",
    "MRK": "Merck & Co., Inc.",
    "JNJ": "Johnson & Johnson",
    "BA": "The Boeing Company",
    "F": "Ford Motor Company",
    "GM": "General Motors Company"
}
 
# We want to ignore references to purely financial or debt-based items
POST_FILTER_KEYWORDS = {
    "notes", "bond", "issuance", "due",
    "fiscal", "earning", "quarter", "result"
}
 
def is_valid_product_post_filter(name: str, desc: str) -> bool:
    """Returns False if name+desc references debt or purely financial data."""
    combined = (name + " " + desc).lower()
    return not any(kw in combined for kw in POST_FILTER_KEYWORDS)
 
def ensure_capital_description(desc: str) -> str:
    """Capitalize the first letter if it starts with a letter."""
    desc = desc.strip()
    if desc and desc[0].isalpha():
        desc = desc[0].upper() + desc[1:]
    return desc
 
def extract_8k_product_info_llm(ticker, headers):
    """
    1) Fetch last 10 8-K filings for 'ticker'.
    2) Retrieve each 8-K, parse text, pass to LLaMA 3.2.
    3) LLM instructed to only return products belonging to 'company_name', ignoring others.
    4) Post-filter references to 'notes' or purely financial data.
    5) Capitalize product descriptions.
    6) Return a DataFrame with columns:
       [Company Name, Stock Name, Filing Time, New Product, Product Description].
    """
 
    df_8ks = get_8k_filings(ticker, headers=headers)
    if df_8ks.empty:
        return pd.DataFrame()
 
    # Sort descending by "filingDate" and limit to 10
    df_8ks = df_8ks.sort_values("filingDate", ascending=False).head(10)
 
    # Map ticker -> real company name
    company_name = TICKER_COMPANY_MAP.get(ticker, ticker)
 
    extracted_data = []
 
    for _, row in df_8ks.iterrows():
        # Build SEC filing URL
        filing_url = (
            f"https://www.sec.gov/Archives/edgar/data/"
            f"{cik_matching_ticker(ticker)}/"
            f"{row['accessionNumber'].replace('-', '')}/"
            f"{row['primaryDocument']}"
        )
 
        resp = requests.get(filing_url, headers=headers)
        if resp.status_code != 200:
            print(f"❌ Failed to fetch {filing_url}")
            continue
 
        # Convert HTML to text
        text = BeautifulSoup(resp.text, "html.parser").get_text(" ", strip=True)
        limited_text = text[:4000]  # Keep it short for LLaMA's context limit
 
        # LLM prompt:
        prompt = f"""
You are analyzing an SEC 8-K filing for {company_name}.
Extract ONLY new products belonging to {company_name}.
Ignore references to other brand names or companies (for example, if you see Tesla but your company is Apple, skip it).
 
Also ignore:
- financial results, earnings, or quarters
- stock splits or executive changes
- acquisitions, intangible assets, or purely financial data
 
Return strictly JSON like:
{{
  "products": [
    {{
      "product_name": "string",
      "product_description": "string"
    }}
  ]
}}
If no product for {company_name}, return {{ "products": [] }}. No extra text or disclaimers.
 
Filing text (truncated):
{limited_text}
"""
 
        llm_response = ollama.chat(
            model="llama3.2",
            messages=[{"role": "user", "content": prompt}]
        )
        llm_output = llm_response["message"]["content"].strip()
 
        print(f"\n🔹 LLaMA OUTPUT from {filing_url}:\n{llm_output}\n")
 
        try:
            data = json.loads(llm_output)
            # Expecting a dict with "products" as a list
            if isinstance(data, dict) and "products" in data:
                products = data["products"]
                if isinstance(products, list):
                    for p in products:
                        prod_name = p.get("product_name", "").strip()
                        prod_desc = p.get("product_description", "").strip()
                        prod_desc = prod_desc[:180]  # limit description length
 
                        if prod_name and is_valid_product_post_filter(prod_name, prod_desc):
                            prod_desc = ensure_capital_description(prod_desc)
                            extracted_data.append({
                                "Company Name": company_name,
                                "Stock Name": ticker,
                                "Filing Time": row["filingDate"],
                                "New Product": prod_name,
                                "Product Description": prod_desc
                            })
                else:
                    print(f"❌ 'products' not a list in JSON for {filing_url}")
            else:
                print(f"❌ Missing 'products' in LLM JSON for {filing_url}")
 
        except json.JSONDecodeError:
            print(f"❌ JSON parsing error for {filing_url}:\n{llm_output}\n")
 
    return pd.DataFrame(extracted_data)
 
if __name__ == "__main__":
    # Updated tickers list to include all companies in TICKER_COMPANY_MAP
    tickers = [
        "AAPL", "MSFT", "TSLA", "AMZN", "GOOGL", "NKE", "NVDA", "LMT",
        "META", "INTC", "AMD", "NFLX", "CRM", "ADBE", "PYPL", "UBER",
        "ROKU", "SBUX", "PFE", "MRK", "JNJ", "BA", "F", "GM"
    ]
 
    all_dfs = []
 
    for t in tickers:
        print(f"\n=== Extracting product info for {t} using LLaMA 3.2 ===\n")
        df_extracted = extract_8k_product_info_llm(t, headers=headers)
        all_dfs.append(df_extracted)
        time.sleep(1)
 
    final_df = pd.concat(all_dfs, ignore_index=True)
    
    # Drop any row where the 'Product Description' is missing, empty, or one of ["string", "none", "null", "no product description provided"] (case-insensitive).
    final_df.dropna(subset=["Product Description"], inplace=True)  # remove rows where it's NaN
    final_df = final_df[final_df["Product Description"].str.strip() != ""]  # remove blank/whitespace-only
    final_df = final_df[~final_df["Product Description"].str.lower().isin(["string", "none", "null", "not specified", "no product description provided", "Not available in filing text"])]
    
    final_df.to_csv("Renan_sp500_8k_extraction.csv", index=False)
    print("\nDone! Check Renan_sp500_8k_extractions.csv")


=== Extracting product info for AAPL using LLaMA 3.2 ===


🔹 LLaMA OUTPUT from https://www.sec.gov/Archives/edgar/data/0000320193/000114036125005876/ef20044022_8k.htm:
{
  "products": [
    {
      "product_name": "AirPods",
      "product_description": ""
    },
    {
      "product_name": "MacBook",
      "product_description": ""
    },
    {
      "product_name": "iPhone",
      "product_description": ""
    }
  ]
}


🔹 LLaMA OUTPUT from https://www.sec.gov/Archives/edgar/data/0000320193/000032019325000007/aapl-20250130.htm:
{
  "products": [
    {
      "product_name": "Apple Watch",
      "product_description": "A smartwatch developed by Apple Inc."
    },
    {
      "product_name": "AirPods",
      "product_description": "Wireless earbuds developed by Apple Inc."
    }
  ]
}


🔹 LLaMA OUTPUT from https://www.sec.gov/Archives/edgar/data/0000320193/000114036125000228/ef20040370_8k.htm:
{
  "products": [
    {
      "product_name": "Apple Watch",
      "product_description": "str